In [105]:
import cv2
import numpy as np
import mahotas
from skimage import color
from matplotlib import pyplot as plt
from skimage import io, color, filters, util
import matplotlib.pyplot as plt


In [136]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import os

# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick



In [129]:
from skimage import measure, io, img_as_bool
import numpy as np
import glob
from cv2 import Sobel
from sklearn import svm, metrics, neighbors

files = glob.glob("Dataset/Motor/Train/*-i/*.jpg")
train_shape_feature = np.zeros((30,3))
for k in range(len(files)):
    BW1 = io.imread(files[k])
    G = util.img_as_ubyte(color.rgb2gray(BW1))
    T = filters.threshold_otsu(G)
    S = util.img_as_float(G < T)
    sf = measure.regionprops(S.astype(int))
    shape_feature = [max(s.moments_hu[0] for s in sf),
    max(s.moments_hu[1] for s in sf),
    max(s.moments_hu[2] for s in sf),]
    train_shape_feature[k,:] = shape_feature
np.savetxt('train_shape_feature.csv', train_shape_feature, delimiter=',', fmt='%f')

files = glob.glob("Dataset/Motor/Test/*-i/*.jp*")
test_shape_feature = np.zeros((15,3))
for k in range(len(files)):
    BW1 = io.imread(files[k])
    G = util.img_as_ubyte(color.rgb2gray(BW1))
    T = filters.threshold_otsu(G)
    S = util.img_as_float(G < T)
    sf = measure.regionprops(S.astype(int))
    shape_feature = [max(s.moments_hu[0] for s in sf),
    max(s.moments_hu[1] for s in sf),
    max(s.moments_hu[2] for s in sf),]
    test_shape_feature[k,:] = shape_feature
np.savetxt('test_shape_feature.csv', test_shape_feature, delimiter=',', fmt='%f')

def accuracy(actual,predicted):
    """This function evaluates the performance of a classification model
    by calculating the Accuracy
    actual = Column matrix with actual class labels of the training
    examples
    predicted = Column matrix with predicted class labels by the
    classification model
    Output: Row matrix with all the performance measures"""
    p = np.count_nonzero(actual == 1)
    n = np.count_nonzero(actual != 1)
    N = p + n
    tp= sum((int(actual[i] == 1 and predicted[i] == 1) for i in range(len(actual))))
    tn = sum((int(actual[i] != 1 and predicted[i] != 1) for i in range(len(actual))))
    fp = n - tn
    fn = p - tp
    accuracy = (tp + tn) / N
    return accuracy

train_label = np.ones((30,1))
train_label[10:20] = 2
train_label[20:] = 3

test_label = np.ones((15,1))
test_label[5:10] = 2
test_label[10:] = 3

train_shape_feature=np.genfromtxt('train_shape_feature.csv',delimiter=',')
test_shape_feature=np.genfromtxt('test_shape_feature.csv',delimiter=',')
 
clf = neighbors.KNeighborsClassifier(3)
clf.fit(train_shape_feature, train_label)
svm_label = clf.predict(test_shape_feature)
cm_svm = metrics.confusion_matrix(svm_label, test_label)
print(cm_svm)

accuracy = accuracy(test_label,svm_label) * 100
print("Akurasi: %f%%" % (accuracy))

c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\skimage\util\dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\skimage\measure\_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\skimage\measure\_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to 

[[5 3 3]
 [0 2 1]
 [0 0 1]]
Akurasi: 60.000000%


c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:65: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [124]:
from skimage import measure, io, img_as_bool
import numpy as np
import glob
from cv2 import Sobel
from sklearn import svm, metrics, neighbors

files = glob.glob("Dataset/Motor/Train/*-i/*.jpg")
train_texture_feature = np.zeros((30,13))
for k in range(len(files)):
    BW1 = io.imread(files[k])

    # extract haralick texture from the image
    features = fd_haralick(BW1)

    train_texture_feature[k,:] = features
np.savetxt('train_texture_feature.csv', train_texture_feature, delimiter=',', fmt='%f')

files = glob.glob("Dataset/Motor/Test/*-i/*.jp*")
test_texture_feature = np.zeros((15,13))
for k in range(len(files)):
    BW1 = io.imread(files[k])

    # extract haralick texture from the image
    features = fd_haralick(BW1)

    test_texture_feature[k,:] = features
np.savetxt('test_texture_feature.csv', test_texture_feature, delimiter=',', fmt='%f')

train_label = np.ones((30,1))
train_label[10:20] = 2
train_label[20:] = 3

test_label = np.ones((15,1))
test_label[5:10] = 2
test_label[10:] = 3

train_texture_feature=np.genfromtxt('train_texture_feature.csv',delimiter=',')
test_texture_feature=np.genfromtxt('test_texture_feature.csv',delimiter=',')
 
clf = neighbors.KNeighborsClassifier(3)
clf.fit(train_texture_feature, train_label)
svm_label = clf.predict(test_texture_feature)
cm_svm = metrics.confusion_matrix(svm_label, test_label)
print(cm_svm)

def accuracy(actual,predicted):
    """This function evaluates the performance of a classification model
    by calculating the Accuracy
    actual = Column matrix with actual class labels of the training
    examples
    predicted = Column matrix with predicted class labels by the
    classification model
    Output: Row matrix with all the performance measures"""
    p = np.count_nonzero(actual == 1)
    n = np.count_nonzero(actual != 1)
    N = p + n
    tp= sum((int(actual[i] == 1 and predicted[i] == 1) for i in range(len(actual))))
    tn = sum((int(actual[i] != 1 and predicted[i] != 1) for i in range(len(actual))))
    fp = n - tn
    fn = p - tp
    accuracy = (tp + tn) / N
    return accuracy

accuracy_1 = accuracy(test_label,svm_label) * 100
print("Akurasi: %f%%" % (accuracy_1))

[[2 2 0]
 [2 2 2]
 [1 1 3]]
Akurasi: 66.666667%


c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:41: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [147]:
from numpy import cov, mean
from numpy.linalg import eig
from sklearn.preprocessing import StandardScaler

def calc_PCA(data) :
    data_for_PCA = StandardScaler().fit_transform(data)
    mean_vec = np.mean(data_for_PCA, axis=0)
    cov_mat = np.cov(data_for_PCA.T)

    eig_vals, eig_vecs = eig(cov_mat)

    # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    exp_var_percentage = 0.97 # Threshold of 97% explained variance

    tot = sum(eig_vals)
    var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
    cum_var_exp = np.cumsum(var_exp)

    num_vec_to_keep = 0

    for index, percentage in enumerate(cum_var_exp):
      if percentage > exp_var_percentage:
        num_vec_to_keep = index + 1
        break

    num_features = data_for_PCA.shape[1]
    proj_mat = eig_pairs[0][1].reshape(num_features,1)
    for eig_vec_idx in range(1, num_vec_to_keep):
      proj_mat = np.hstack((proj_mat, eig_pairs[eig_vec_idx][1].reshape(num_features,1)))

    # Project the data 
    pca_data = data_for_PCA.dot(proj_mat)

    return(pca_data)

In [164]:
train_feature_1_PCA = calc_PCA(train_shape_feature)
test_feature_1_PCA = calc_PCA(test_shape_feature)

train_feature_2_PCA = calc_PCA(train_texture_feature)
test_feature_2_PCA = calc_PCA(test_texture_feature)

train_feature_combined = np.hstack((train_feature_1_PCA,train_feature_2_PCA))
test_feature_combined = np.hstack((test_feature_1_PCA,test_feature_2_PCA))

train_label = np.ones((30,1))
train_label[10:20] = 2
train_label[20:] = 3

test_label = np.ones((15,1))
test_label[5:10] = 2
test_label[10:] = 3

classifier = neighbors.KNeighborsClassifier(3)
classifier.fit(train_feature_combined, train_label)

knn_label = classifier.predict(test_feature_combined)
conf_mat = metrics.confusion_matrix(knn_label,test_label)
print(conf_mat)
accuracy_1 = accuracy(test_label,knn_label) * 100
print("Akurasi: %f%%" % (accuracy_1))

[[2 1 0]
 [1 3 1]
 [2 1 4]]
Akurasi: 73.333333%


c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [166]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# create all the machine learning models
num_trees = 100
test_size = 0.10
seed      = 9
models = []
models.append(('LR', LogisticRegression(random_state=seed)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier(random_state=seed)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, random_state=seed)))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(random_state=seed)))

def accuracy(actual,predicted):
    """This function evaluates the performance of a classification model
    by calculating the Accuracy
    actual = Column matrix with actual class labels of the training
    examples
    predicted = Column matrix with predicted class labels by the
    classification model
    Output: Row matrix with all the performance measures"""
    p = np.count_nonzero(actual == 1)
    n = np.count_nonzero(actual != 1)
    N = p + n
    tp= sum((int(actual[i] == 1 and predicted[i] == 1) for i in range(len(actual))))
    tn = sum((int(actual[i] != 1 and predicted[i] != 1) for i in range(len(actual))))
    fp = n - tn
    fn = p - tp
    accuracy = (tp + tn) / N
    return accuracy

for name, model in models:
    train_label = np.ones((30,1))
    train_label[10:20] = 2
    train_label[20:] = 3

    test_label = np.ones((15,1))
    test_label[5:10] = 2
    test_label[10:] = 3
    train_shape_feature=np.genfromtxt('train_shape_feature.csv',delimiter=',')
    test_shape_feature=np.genfromtxt('test_shape_feature.csv',delimiter=',')    
    
    clf = model
    clf.fit(train_shape_feature, train_label)
    svm_label = clf.predict(test_shape_feature)
    cm_svm = metrics.confusion_matrix(svm_label, test_label)
    print(cm_svm)
    accuracy_1 = accuracy(test_label,svm_label) * 100
    print("Akurasi " + name + " Shape: %f" % (accuracy_1))
    
    train_label = np.ones((30,1))
    train_label[10:20] = 2
    train_label[20:] = 3

    test_label = np.ones((15,1))
    test_label[5:10] = 2
    test_label[10:] = 3
    train_texture_feature=np.genfromtxt('train_texture_feature.csv',delimiter=',')
    test_texture_feature=np.genfromtxt('test_texture_feature.csv',delimiter=',')
 
    clf = model
    clf.fit(train_texture_feature, train_label)
    svm_label = clf.predict(test_texture_feature)
    cm_svm = metrics.confusion_matrix(svm_label, test_label)
    print(cm_svm)
    accuracy_1 = accuracy(test_label,svm_label) * 100
    print("Akurasi " + name + " Texture: %f" % (accuracy_1))
    
    train_label = np.ones((30,1))
    train_label[10:20] = 2
    train_label[20:] = 3

    test_label = np.ones((15,1))
    test_label[5:10] = 2
    test_label[10:] = 3
    
    knn_label = classifier.predict(test_feature_combined)
    conf_mat = metrics.confusion_matrix(knn_label,test_label)
    accuracy_1 = accuracy(test_label,knn_label) * 100
    print(conf_mat)
    print("Akurasi " + name + " PCA: %f" % (accuracy_1))
    print()

c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed t

[[0 0 0]
 [1 4 2]
 [4 1 3]]
Akurasi LR Shape: 66.666667
[[0 1 0]
 [3 4 3]
 [2 0 2]]
Akurasi LR Texture: 60.000000
[[2 1 0]
 [1 3 1]
 [2 1 4]]
Akurasi LR PCA: 73.333333

[[1 4 1]
 [1 0 0]
 [3 1 4]]
Akurasi LDA Shape: 40.000000
[[1 1 1]
 [3 4 1]
 [1 0 3]]
Akurasi LDA Texture: 60.000000
[[2 1 0]
 [1 3 1]
 [2 1 4]]
Akurasi LDA PCA: 73.333333

[[2 3 1]
 [2 2 2]
 [1 0 2]]
Akurasi KNN Shape: 53.333333
[[3 1 1]
 [2 3 2]
 [0 1 2]]
Akurasi KNN Texture: 73.333333
[[2 1 0]
 [1 3 1]
 [2 1 4]]
Akurasi KNN PCA: 73.333333

[[3 2 0]
 [2 2 2]
 [0 1 3]]
Akurasi CART Shape: 73.333333
[[1 0 0]
 [3 5 3]
 [1 0 2]]
Akurasi CART Texture: 73.333333
[[2 1 0]
 [1 3 1]
 [2 1 4]]
Akurasi CART PCA: 73.333333

[[4 2 0]
 [0 3 1]
 [1 0 4]]
Akurasi RF Shape: 80.000000
[[1 1 2]
 [2 3 1]
 [2 1 2]]
Akurasi RF Texture: 53.333333
[[2 1 0]
 [1 3 1]
 [2 1 4]]
Akurasi RF PCA: 73.333333

[[1 2 2]
 [0 1 1]
 [4 2 2]]
Akurasi NB Shape: 46.666667
[[1 3 0]
 [2 1 3]
 [2 1 2]]
Akurasi NB Texture: 53.333333
[[2 1 0]
 [1 3 1]
 [2 1 4]]
A

c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\sk